In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import radiomics
from radiomics import featureextractor
import os
import xml.etree.ElementTree as ET
import SimpleITK as sitk
from __future__ import print_function
import logging
import csv
import shutil


In [3]:

def move_and_rename_xml_files(base_dir, folders, destination_dir):
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    file_counter = 1

    # Iterar pelas pastas originais
    for folder in folders:
        folder_path = os.path.join(base_dir, folder)

        # Iterar pelos arquivos na pasta
        for xml_file in os.listdir(folder_path):
            if xml_file.endswith(".xml"):
                source_path = os.path.join(folder_path, xml_file)
                destination_path = os.path.join(destination_dir, f"{file_counter}.xml")
                
                # Mover e renomear o arquivo
                shutil.move(source_path, destination_path)
                print(f"Arquivo {xml_file} movido e renomeado para {file_counter}.xml")
                
                file_counter += 1
    print("Concluido com sucesso!")

# Diretório base onde estão as pastas
base_dir = "LIDC_XML_only/tcia-lidc-xml"
folders = ['157', '185', '186', '187', '188', '189']

# Diretório de destino para os arquivos renomeados
destination_dir = "renamed_files"

# Chamar a função para mover e renomear os arquivos
move_and_rename_xml_files(base_dir, folders, destination_dir)

NameError: name 'os' is not defined

In [16]:

def extract_uids_from_xml(xml_file):
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        # Extrair StudyInstanceUID e SeriesInstanceUID
        ns = {'ns': 'http://www.nih.gov'}  # Namespace no XML
        study_uid_element = root.find('.//ns:StudyInstanceUID', ns)
        series_uid_element = root.find('.//ns:SeriesInstanceUid', ns)
        
        if study_uid_element is None or series_uid_element is None:
            return None, None  # Ignorar arquivos sem as tags
        
        study_uid = study_uid_element.text
        series_uid = series_uid_element.text
        
        return study_uid, series_uid
    except ET.ParseError as e:
        print(f"Erro ao analisar o arquivo XML {xml_file}: {e}")
        return None, None  # Retorna None se houver erro ao processar o XML

def iterate_xml_files_and_store_uids_csv(base_dir, output_file):
    # Abrir arquivo CSV para escrita
    with open(output_file, mode='w', newline='') as csv_file:
        fieldnames = ['Arquivo', 'StudyInstanceUID', 'SeriesInstanceUID']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        # Escrever o cabeçalho
        writer.writeheader()
        
        # Iterar pelos arquivos XML na pasta
        for xml_file in os.listdir(base_dir):  # Usa base_dir aqui
            if xml_file.endswith(".xml"):
                xml_path = os.path.join(base_dir, xml_file)
                
                study_uid, series_uid = extract_uids_from_xml(xml_path)
                
                # Só escrever se os UIDs foram encontrados
                if study_uid and series_uid:
                    writer.writerow({
                        'Arquivo': xml_file,
                        'StudyInstanceUID': study_uid,
                        'SeriesInstanceUID': series_uid
                    })

# Diretório base onde estão os arquivos renomeados
base_dir = "LIDC_XML_only/renamed_files"
output_file = "uids.csv"

# Chamar a função para iterar e guardar os UIDs em um CSV
iterate_xml_files_and_store_uids_csv(base_dir, output_file)

print(f"UIDs salvos em {output_file}")

UIDs salvos em uids.csv


mascara dos arquivos xml

In [17]:

def create_mask_from_xml(dicom_image, xml_file):
    # Ler o arquivo XML
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # Obter as dimensões da imagem DICOM
    size = dicom_image.GetSize()  # (Width, Height, Depth) para 3D ou (Width, Height) para 2D
    if len(size) == 3:
        mask = np.zeros(size, dtype=np.uint8)  # Máscara 3D
    else:
        mask = np.zeros((size[1], size[0]), dtype=np.uint8)  # Máscara 2D
    
    # Processar o XML para extrair as coordenadas dos nódulos
    for lesion in root.findall('.//lesion'):
        for roi in lesion.findall('.//roi'):
            for edgeMap in roi.findall('.//edgeMap'):
                x = int(edgeMap.find('xCoord').text)
                y = int(edgeMap.find('yCoord').text)
                
                # Verificar se é 2D ou 3D (DICOM volume)
                if len(size) == 3:
                    z = int(roi.find('imageZposition').text)  # Exemplo de Z vindo do XML
                    mask[y, x, z] = 1  # Nota: y primeiro, depois x, depois z
                else:
                    mask[y, x] = 1  # Para imagens 2D

    # Converter o numpy array para uma imagem SimpleITK
    mask_image = sitk.GetImageFromArray(mask)
    
    # Configurar a geometria da máscara para ser a mesma da imagem DICOM
    mask_image.CopyInformation(dicom_image)
    
    return mask_image

In [18]:

def create_mask_from_xml(dicom_image, xml_file):
    # Função para criar máscara do XML (similar à função que você postou antes)
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    size = dicom_image.GetSize()
    mask = np.zeros(size, dtype=np.uint8)
    
    for lesion in root.findall('.//lesion'):
        for roi in lesion.findall('.//roi'):
            for edgeMap in roi.findall('.//edgeMap'):
                x = int(edgeMap.find('xCoord').text)
                y = int(edgeMap.find('yCoord').text)
                mask[y, x] = 1
    
    mask_image = sitk.GetImageFromArray(mask)
    mask_image.CopyInformation(dicom_image)
    
    return mask_image

In [19]:
def create_mask_from_xml(dicom_image, xml_file):
    # Função para criar máscara do XML (similar à função que você postou antes)
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    size = dicom_image.GetSize()
    mask = np.zeros(size, dtype=np.uint8)
    
    for lesion in root.findall('.//lesion'):
        for roi in lesion.findall('.//roi'):
            for edgeMap in roi.findall('.//edgeMap'):
                x = int(edgeMap.find('xCoord').text)
                y = int(edgeMap.find('yCoord').text)
                mask[y, x] = 1
    
    mask_image = sitk.GetImageFromArray(mask)
    mask_image.CopyInformation(dicom_image)
    
    return mask_image

In [6]:

def feature_extraction():
    base_dir = r'pre_processing/LIDC-IDRI-files'  # Pasta com subpastas de pacientes
    outPath = r'pre_processing/feature_extraction.csv'  # Caminho de saída
    xml_csv_path = r'pre_processing/uids.csv'  # Caminho para o CSV com os UIDs

    outputFilepath = os.path.join(outPath, 'radiomics_features.csv')
    progress_filename = os.path.join(outPath, 'pyrad_log.txt')
    params = os.path.join(outPath, 'exampleSettings', 'Params.yaml')

    # Configurar logging
    rLogger = logging.getLogger('radiomics')
    handler = logging.FileHandler(filename=progress_filename, mode='w')
    handler.setFormatter(logging.Formatter('%(levelname)s:%(name)s: %(message)s'))
    rLogger.addHandler(handler)
    logger = rLogger.getChild('batch')
    radiomics.setVerbosity(logging.INFO)
    logger.info('Versão do PyRadiomics: %s', radiomics.__version__)

    # Carregar parâmetros
    if os.path.isfile(params):
        extractor = featureextractor.RadiomicsFeatureExtractor(params)
    else:
        settings = {
            'binWidth': 25,
            'resampledPixelSpacing': None,
            'interpolator': sitk.sitkBSpline,
            'enableCExtensions': True
        }
        extractor = featureextractor.RadiomicsFeatureExtractor(**settings)

    logger.info('Tipos de imagem habilitados: %s', extractor.enabledImagetypes)
    logger.info('Features habilitadas: %s', extractor.enabledFeatures)
    logger.info('Configurações atuais: %s', extractor.settings)

    # Carregar o CSV com os UIDs
    uids_df = pd.read_csv(xml_csv_path)

    # Inicializar um DataFrame para armazenar os resultados
    results = pd.DataFrame()

    # Iterar pelas subpastas dos pacientes
    for patient_folder in os.listdir(base_dir):
        patient_path = os.path.join(base_dir, patient_folder)
        
        if os.path.isdir(patient_path):
            dicom_files = [os.path.join(patient_path, f) for f in os.listdir(patient_path) if f.endswith('.dcm')]

            if len(dicom_files) > 0:
                imageFilepath = dicom_files[0]  # Usando o primeiro DICOM da pasta

                # Encontrar o XML correspondente no CSV
                dicom_metadata = sitk.ReadImage(imageFilepath)
                study_uid = dicom_metadata.GetMetaData("0020|000D")  # StudyInstanceUID do DICOM
                series_uid = dicom_metadata.GetMetaData("0020|000E")  # SeriesInstanceUID do DICOM

                xml_row = uids_df[(uids_df['StudyInstanceUID'] == study_uid) & 
                                  (uids_df['SeriesInstanceUID'] == series_uid)]
                
                if not xml_row.empty:
                    xml_file = xml_row['Arquivo'].values[0]
                    xml_file_path = os.path.join(base_dir, 'path_to_xml_files', xml_file)

                    # Criar a máscara a partir do XML
                    mask_image = create_mask_from_xml(dicom_metadata, xml_file_path)

                    logger.info("Processando Paciente: %s, Imagem: %s, Máscara: %s", 
                                patient_folder, os.path.basename(imageFilepath), os.path.basename(xml_file))

                    try:
                        # Executar a extração de features
                        result = pd.Series(extractor.execute(imageFilepath, mask_image))
                        result['PatientID'] = patient_folder
                        result['Image'] = os.path.basename(imageFilepath)
                        result['Mask'] = os.path.basename(xml_file)
                        
                        # Adicionar as features ao DataFrame de resultados
                        results = results.append(result, ignore_index=True)
                        
                    except Exception:
                        logger.error('FALHA NA EXTRAÇÃO DE FEATURES:', exc_info=True)

    logger.info('Extração completa, escrevendo CSV')
    results.to_csv(outputFilepath, index=False, na_rep='NaN')
    logger.info('Gravação do CSV completa')

feature_extraction()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/gabrielasimon/Desktop/lab-iacd/pre_processing/pre_processing/feature_extraction.csv/pyrad_log.txt'

In [7]:


def feature_extraction():
    base_dir = r'pre_processing/LIDC-IDRI-files'  # Pasta com subpastas de pacientes
    outPath = r'pre_processing/feature_extraction.csv'  # Caminho de saída
    xml_csv_path = r'pre_processing/uids.csv'  # Caminho para o CSV com os UIDs

    outputFilepath = os.path.join(outPath, 'radiomics_features.csv')
    progress_filename = os.path.join(outPath, 'pyrad_log.txt')
    params = os.path.join(outPath, 'exampleSettings', 'Params.yaml')

    # Configurar logging
    rLogger = logging.getLogger('radiomics')
    handler = logging.FileHandler(filename=progress_filename, mode='w')
    handler.setFormatter(logging.Formatter('%(levelname)s:%(name)s: %(message)s'))
    rLogger.addHandler(handler)
    logger = rLogger.getChild('batch')
    radiomics.setVerbosity(logging.INFO)
    logger.info('Versão do PyRadiomics: %s', radiomics.__version__)

    # Carregar parâmetros
    if os.path.isfile(params):
        extractor = featureextractor.RadiomicsFeatureExtractor(params)
    else:
        settings = {
            'binWidth': 25,
            'resampledPixelSpacing': None,
            'interpolator': sitk.sitkBSpline,
            'enableCExtensions': True
        }
        extractor = featureextractor.RadiomicsFeatureExtractor(**settings)

    logger.info('Tipos de imagem habilitados: %s', extractor.enabledImagetypes)
    logger.info('Features habilitadas: %s', extractor.enabledFeatures)
    logger.info('Configurações atuais: %s', extractor.settings)

    # Carregar o CSV com os UIDs
    uids_df = pd.read_csv(xml_csv_path)

    # Inicializar um DataFrame para armazenar os resultados
    results = pd.DataFrame()

    # Iterar pelas subpastas dos pacientes
    for patient_folder in os.listdir(base_dir):
        patient_path = os.path.join(base_dir, patient_folder)
        
        if os.path.isdir(patient_path):
            dicom_files = [os.path.join(patient_path, f) for f in os.listdir(patient_path) if f.endswith('.dcm')]

            if len(dicom_files) > 0:
                imageFilepath = dicom_files[0]  # Usando o primeiro DICOM da pasta

                # Encontrar o XML correspondente no CSV
                dicom_metadata = sitk.ReadImage(imageFilepath)
                study_uid = dicom_metadata.GetMetaData("0020|000D")  # StudyInstanceUID do DICOM
                series_uid = dicom_metadata.GetMetaData("0020|000E")  # SeriesInstanceUID do DICOM

                xml_row = uids_df[(uids_df['StudyInstanceUID'] == study_uid) & 
                                  (uids_df['SeriesInstanceUID'] == series_uid)]
                
                if not xml_row.empty:
                    xml_file = xml_row['Arquivo'].values[0]
                    xml_file_path = os.path.join(base_dir, 'path_to_xml_files', xml_file)

                    try:
                        # Criar a máscara a partir do XML
                        mask_image = create_mask_from_xml(dicom_metadata, xml_file_path)

                        logger.info("Processando Paciente: %s, Imagem: %s, Máscara: %s", 
                                    patient_folder, os.path.basename(imageFilepath), os.path.basename(xml_file))

                        try:
                            # Executar a extração de features
                            result = pd.Series(extractor.execute(imageFilepath, mask_image))
                            result['PatientID'] = patient_folder
                            result['Image'] = os.path.basename(imageFilepath)
                            result['Mask'] = os.path.basename(xml_file)
                            
                            # Adicionar as features ao DataFrame de resultados
                            results = results.append(result, ignore_index=True)
                            
                        except Exception:
                            logger.error('FALHA NA EXTRAÇÃO DE FEATURES:', exc_info=True)

                    except Exception as e:
                        logger.error(f"Erro ao criar a máscara para o paciente {patient_folder}: {e}")

                else:
                    logger.warning(f"XML correspondente não encontrado para o paciente {patient_folder}. Pulando.")

    logger.info('Extração completa, escrevendo CSV')
    results.to_csv(outputFilepath, index=False, na_rep='NaN')
    logger.info('Gravação do CSV completa')

feature_extraction()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/gabrielasimon/Desktop/lab-iacd/pre_processing/pre_processing/feature_extraction.csv/pyrad_log.txt'

In [ ]:

def feature_extraction():
    base_dir = r'pre_processing/LIDC-IDRI-files'  # Pasta com subpastas de pacientes
    outPath = r'pre_processing/feature_extraction'  # Corrigido: deve ser um diretório
    xml_csv_path = r'pre_processing/uids.csv'  # Caminho para o CSV com os UIDs

    # Certifique-se de que o diretório de saída existe
    if not os.path.exists(outPath):
        os.makedirs(outPath)

    outputFilepath = os.path.join(outPath, 'radiomics_features.csv')
    progress_filename = os.path.join(outPath, 'pyrad_log.txt')
    params = os.path.join(outPath, 'exampleSettings', 'Params.yaml')

    # Configurar logging
    rLogger = logging.getLogger('radiomics')
    handler = logging.FileHandler(filename=progress_filename, mode='w')
    handler.setFormatter(logging.Formatter('%(levelname)s:%(name)s: %(message)s'))
    rLogger.addHandler(handler)
    logger = rLogger.getChild('batch')
    featureextractor.setVerbosity(logging.INFO)
    logger.info('Versão do PyRadiomics: %s', featureextractor.__version__)

    # Carregar parâmetros
    if os.path.isfile(params):
        extractor = featureextractor.RadiomicsFeatureExtractor(params)
    else:
        settings = {
            'binWidth': 25,
            'resampledPixelSpacing': None,
            'interpolator': sitk.sitkBSpline,
            'enableCExtensions': True
        }
        extractor = featureextractor.RadiomicsFeatureExtractor(**settings)

    logger.info('Tipos de imagem habilitados: %s', extractor.enabledImagetypes)
    logger.info('Features habilitadas: %s', extractor.enabledFeatures)
    logger.info('Configurações atuais: %s', extractor.settings)

    # Carregar o CSV com os UIDs
    uids_df = pd.read_csv(xml_csv_path)

    # Inicializar um DataFrame para armazenar os resultados
    results = pd.DataFrame()

    # Iterar pelas subpastas dos pacientes
    for patient_folder in os.listdir(base_dir):
        patient_path = os.path.join(base_dir, patient_folder)
        
        if os.path.isdir(patient_path):
            dicom_files = [os.path.join(patient_path, f) for f in os.listdir(patient_path) if f.endswith('.dcm')]

            if len(dicom_files) > 0:
                imageFilepath = dicom_files[0]  # Usando o primeiro DICOM da pasta

                # Encontrar o XML correspondente no CSV
                dicom_metadata = sitk.ReadImage(imageFilepath)
                study_uid = dicom_metadata.GetMetaData("0020|000D")  # StudyInstanceUID do DICOM
                series_uid = dicom_metadata.GetMetaData("0020|000E")  # SeriesInstanceUID do DICOM

                xml_row = uids_df[(uids_df['StudyInstanceUID'] == study_uid) & 
                                  (uids_df['SeriesInstanceUID'] == series_uid)]
                
                if not xml_row.empty:
                    xml_file = xml_row['Arquivo'].values[0]
                    xml_file_path = os.path.join(base_dir, 'path_to_xml_files', xml_file)

                    try:
                        # Criar a máscara a partir do XML
                        mask_image = create_mask_from_xml(dicom_metadata, xml_file_path)

                        logger.info("Processando Paciente: %s, Imagem: %s, Máscara: %s", 
                                    patient_folder, os.path.basename(imageFilepath), os.path.basename(xml_file))

                        try:
                            # Executar a extração de features
                            result = pd.Series(extractor.execute(imageFilepath, mask_image))
                            result['PatientID'] = patient_folder
                            result['Image'] = os.path.basename(imageFilepath)
                            result['Mask'] = os.path.basename(xml_file)
                            
                            # Adicionar as features ao DataFrame de resultados
                            results = results.append(result, ignore_index=True)
                            
                        except Exception:
                            logger.error('FALHA NA EXTRAÇÃO DE FEATURES:', exc_info=True)

                    except Exception as e:
                        logger.error(f"Erro ao criar a máscara para o paciente {patient_folder}: {e}")

                else:
                    logger.warning(f"XML correspondente não encontrado para o paciente {patient_folder}. Pulando.")

    logger.info('Extração completa, escrevendo CSV')
    results.to_csv(outputFilepath, index=False, na_rep='NaN')
    logger.info('Gravação do CSV completa')

feature_extraction()